In [32]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
import glob
import xml.etree.ElementTree as ET
import h5py
import warnings
import re


# from data_generator.object_detection_2d_data_generator import DataGenerator

In [69]:
warnings.filterwarnings('ignore')

#входное разрешение модели
IMAGE_HEIGHT = 28 # Height of the input images
IMAGE_WIDTH = 28 #Width of the input images
WORK_DIR = 'D:\work\\test_comp_vision\datasets\!_lines_w25_parsed_test' #путь к основной папке
LABELS = '0123456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [62]:
"""
Структура папок:
base_folder
    image_0
        image_0_0.jpg
        image_0_0.txt
        image_0_1.jpg
        image_0_1.txt
        ...
    image_1
        image_1_0.jpg
        image_1_0.txt
        image_1_1.jpg
        image_1_1.txt
        ...
        
"""
# TODO заменить работу с индексами над путями к файлам на нормальную обработку разрешения файлов
def get_dataframe(path):
    # pattern = r'/^(.+)(\.[^ .]+)?$/'      # для удаления разрешения из пути к файлу
    
    symbol_classes = []
    for folder in os.listdir(path):

        for img_file in glob.glob(f'{path}/{folder}/*.jpg'):
            txt_file =  f"{img_file[:-3]}txt"    # меняем разрешение для открытия текста [:-3]
            # print(f'IMG_file: {img_file}\nTXT_file: {txt_file}')
            with open(txt_file, 'r') as f:
                symbol_class = f.read()
                
            value = (img_file, symbol_class)
            symbol_classes.append(value)
    column_names = ['file_name', 'class']
    return pd.DataFrame(symbol_classes, columns=column_names)

In [64]:
#Создаем DataFrame в котором хранится путь к фалу/изображению и его класс
base_path = WORK_DIR #путь к основной папке
df_text = pd.DataFrame()
df_text = df_text.append(get_dataframe(base_path))
df_text

,file_name,class
0,D:\work\test_comp_vision\datasets\!_lines_w25_...,в
1,D:\work\test_comp_vision\datasets\!_lines_w25_...,е
2,D:\work\test_comp_vision\datasets\!_lines_w25_...,е
3,D:\work\test_comp_vision\datasets\!_lines_w25_...,р
4,D:\work\test_comp_vision\datasets\!_lines_w25_...,а
...,...,...
190,D:\work\test_comp_vision\datasets\!_lines_w25_...,С
191,D:\work\test_comp_vision\datasets\!_lines_w25_...,Я
192,D:\work\test_comp_vision\datasets\!_lines_w25_...,Ч
193,D:\work\test_comp_vision\datasets\!_lines_w25_...,Т


In [70]:
# Преобразуем классы к числовым значениям
labels_to_int = {}
for i, label in enumerate(LABELS):
    labels_to_int[label] = i
print(labels_to_int)

df_img = pd.DataFrame({'image_name':df_text['file_name'],
                   'class_id':[labels_to_int[label] for label in df_text['class']]})

#df_img.to_csv(WORK_DIR, index=False)
df_img

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'А': 10, 'Б': 11, 'В': 12, 'Г': 13, 'Д': 14, 'Е': 15, 'Ё': 16, 'Ж': 17, 'З': 18, 'И': 19, 'Й': 20, 'К': 21, 'Л': 22, 'М': 23, 'Н': 24, 'О': 25, 'П': 26, 'Р': 27, 'С': 28, 'Т': 29, 'У': 30, 'Ф': 31, 'Х': 32, 'Ц': 33, 'Ч': 34, 'Ш': 35, 'Щ': 36, 'Ъ': 37, 'Ы': 38, 'Ь': 39, 'Э': 40, 'Ю': 41, 'Я': 42, 'а': 43, 'б': 44, 'в': 45, 'г': 46, 'д': 47, 'е': 48, 'ё': 49, 'ж': 50, 'з': 51, 'и': 52, 'й': 53, 'к': 54, 'л': 55, 'м': 56, 'н': 57, 'о': 58, 'п': 59, 'р': 60, 'с': 61, 'т': 62, 'у': 63, 'ф': 64, 'х': 65, 'ц': 66, 'ч': 67, 'ш': 68, 'щ': 69, 'ъ': 70, 'ы': 71, 'ь': 72, 'э': 73, 'ю': 74, 'я': 75}


,image_name,class_id
0,D:\work\test_comp_vision\datasets\!_lines_w25_...,45
1,D:\work\test_comp_vision\datasets\!_lines_w25_...,48
2,D:\work\test_comp_vision\datasets\!_lines_w25_...,48
3,D:\work\test_comp_vision\datasets\!_lines_w25_...,60
4,D:\work\test_comp_vision\datasets\!_lines_w25_...,43
...,...,...
190,D:\work\test_comp_vision\datasets\!_lines_w25_...,28
191,D:\work\test_comp_vision\datasets\!_lines_w25_...,42
192,D:\work\test_comp_vision\datasets\!_lines_w25_...,34
193,D:\work\test_comp_vision\datasets\!_lines_w25_...,29


In [11]:
len(pd.unique(df['image_name']))

2706

In [12]:
#Создаем файл датасета для обучения детектора
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
images_dir = ''

train_labels_filename = 'data/ZAVOD.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

train_dataset.create_hdf5_dataset(file_path='data/dataset_zavod_288x512.h5',
                                  resize=[img_height,img_width],
                                  variable_image_size=True,
                                  verbose=True)

Creating HDF5 dataset: 100%|██████████| 2706/2706 [00:48<00:00, 55.91it/s]
